### Load model created by analyse_docs(...)

In [25]:
import os.path
import fasttext
import pickle as pkl
import shutup

from huggingface_hub import hf_hub_download
from bertopic import BERTopic
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

shutup.please()

base_path = "/home/s371513/ernie/"

topic_model = BERTopic.load(os.path.join(base_path, "model"))
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

tmodel = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")



### Label Topics

In [27]:
custom_topics = {}

for i in range(len(topic_model.get_topic_info())):
    s = ""
    
    for tup in topic_model.get_topic(i):    
        try:
            impact_factor = float(tup[1])
            topic_keyword = tup[0]
        except Exception as _:
            impact_factor = float(tup[0])
            topic_keyword = tup[1]
            
        encoded_hi = tokenizer(topic_keyword, return_tensors="pt")
        generated_tokens = tmodel.generate(**encoded_hi)
        token = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        s += str(token) + " [" + str(impact_factor) + "]\n"
    
    topic = input("----------------------------\nTopics already created: \n{}\n----------Topic {}----------\n{}----------------------------".format(set(custom_topics.values()), str(i), s))
        
    custom_topics[i] = topic

with open(os.path.join(base_path, "custom_topics.pkl"), "wb+") as f:
    pkl.dump(custom_topics, f)


KeyboardInterrupt



### Match Topics

In [ ]:
    if os.path.isfile(os.path.join(cfg.base_path(), "custom_topics.pkl")):
        with open(os.path.join(cfg.base_path(), "custom_topics.pkl"), 'rb') as f:
            print("Loading custom topics...")
            topic_dictionary = pkl.load(f)

            print("Reading texts...")
            file_names, _ = read_text("preprocessed.txt")

            print("Applying topics to files in batches of {}...".format(chunk_size))
            file_chunks = [file_names[i:i + chunk_size] for i in range(0, len(file_names), chunk_size)]

            c = 1
            for file_chunk in file_chunks:
                print("-" * 16)
                print("Batch {} of {}".format(c, len(file_chunks)))
                print("-" * 16)
                loop = tqdm(file_chunk)

                corrupted_files = 0

                for topic_index, doc_file in enumerate(loop):
                    with open(os.path.join(cfg.gdelt_out(), doc_file), "rb") as d:
                        try:
                            document = pkl.load(d)
                            d.close()
                            document.topic_information = topic_dictionary[topic_index]
                            document.save_document()

                        except EOFError as _:
                            corrupted_files += 1
                            loop.set_postfix_str("Coruppted files: {}".format(corrupted_files))

                        d.close()
                c += 1
    else:
        print("No file of custom topics found, execute label_topics() first!")